In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# %cd /content/drive/MyDrive/Colab Notebooks/Dacon/samsung_ai/data/

# !unzip -qq "/content/drive/MyDrive/Colab Notebooks/Dacon/samsung_ai/data/data.zip"

/content/drive/MyDrive/Colab Notebooks/Dacon/samsung_ai/data


In [3]:
! pip install rdkit
!pip install transformers datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 36.8 MB 50.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 4.0 MB/s 
     |████████████████████████████████| 365 kB 71.5 MB/s 
     |████████████████████████████████| 6.6 MB 39.1 MB/s 
     |████████████████████████████████| 120 kB 49.9 MB/s 
     |████████████████████████████████| 212 kB 64.5 MB/s 
     |████████████████████████████████| 115 kB 66.5 MB/s 
     |████████████████████████████████| 127 kB 49.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [4]:
import pandas as pd
import numpy as np
import random
import os

from rdkit import Chem 
from tqdm import tqdm

import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AdamWeightDecay, AutoModelForSequenceClassification,AutoModel, AutoConfig, AutoTokenizer, TrainingArguments, Trainer

In [5]:
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore

seed_everything()

In [6]:
import os
path2 = '/content/drive/MyDrive/Colab Notebooks/Dacon/samsung_ai/data/data/mol_files/'

print(os.path)

<module 'posixpath' from '/usr/lib/python3.7/posixpath.py'>


In [6]:
# from glob import glob
# import re
# train_mol = sorted(glob(path +"train_set/*.mol"))

In [25]:
from rdkit.Chem import AllChem
from rdkit import Chem, DataStructs
class ECFP6:
    def __init__(self, smiles):
        self.mols = [Chem.MolFromSmiles(i) for i in smiles]
        self.smiles = smiles

    def mol2fp(self, mol, radius = 5):
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, nBits = 2048,radius = radius)
        array = np.zeros((1,))
        DataStructs.ConvertToNumpyArray(fp, array)
        return array

    def compute_ECFP6(self, name):
        bit_headers = ['bit' + str(i) for i in range(2048)]
        arr = np.empty((0,2048), int).astype(int)
        for i in self.mols:
            fp = self.mol2fp(i)
            arr = np.vstack((arr, fp))
        df_ecfp6 = pd.DataFrame(np.asarray(arr).astype(int),columns=bit_headers)
        df_ecfp6.insert(loc=0, column='smiles', value=self.smiles)
        df_ecfp6.to_csv(path2 + name +'_ECFP6.csv', index=False)

In [8]:
path = '/content/drive/MyDrive/Colab Notebooks/Dacon/samsung_ai/data/'

In [9]:
train = pd.read_csv(path + 'train_set.ReorgE.csv', index_col = 0)
test = pd.read_csv(path + 'test_set.csv', index_col = 0)
submission = pd.read_csv(path + 'sample_submission.csv', index_col = 0)

In [26]:
ecfp6_descriptor = ECFP6(train.SMILES)
ecfp6_descriptor.compute_ECFP6('train2048_radius5')

In [27]:
ecfp6_descriptor2 = ECFP6(test.SMILES)
ecfp6_descriptor2.compute_ECFP6('test2048_radius5')

In [48]:
train_ecfp = pd.read_csv(path2 + 'train_ECFP6.csv')
test_ecfp = pd.read_csv(path2 + 'test_ECFP6.csv')

In [49]:
train_x = train_ecfp.iloc[:,1:]
train_y = train[['Reorg_g','Reorg_ex']]
#train_y_ex = train[['Reorg_g','Reorg_ex']]
test_x = test_ecfp.iloc[:,1:]

In [22]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor, StackingRegressor
import lightgbm as lgb
LR_g_1 = lgb.LGBMRegressor(random_state=42, n_estimators=1000, max_depth=61, learning_rate=0.075)
LR_g_2 = lgb.LGBMRegressor(random_state=22, n_estimators=1000, max_depth=61, learning_rate=0.075)
LR_g_3 = lgb.LGBMRegressor(random_state=2, n_estimators=1000, max_depth=61, learning_rate=0.075)


estimators = [     ('lr1', LR_g_1),
    ('lr2', LR_g_2),
     ('lr3', LR_g_3)

]
reg = StackingRegressor(
    estimators=estimators,
     #final_estimator=RandomForestRegressor(n_estimators=10,
                                           #random_state=42)
)
reg.fit(train_x, train_y_g)

StackingRegressor(estimators=[('lr1',
                               LGBMRegressor(learning_rate=0.075, max_depth=61,
                                             n_estimators=1000,
                                             random_state=42)),
                              ('lr2',
                               LGBMRegressor(learning_rate=0.075, max_depth=61,
                                             n_estimators=1000,
                                             random_state=22)),
                              ('lr3',
                               LGBMRegressor(learning_rate=0.075, max_depth=61,
                                             n_estimators=1000,
                                             random_state=2))])

In [23]:
test_g = reg.predict(test_x)

In [24]:
test_g.max()

0.8802740546565183

In [16]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor, StackingRegressor
import lightgbm as lgb
LR_ex_1 = lgb.LGBMRegressor(random_state=42, n_estimators=1000, max_depth=61, learning_rate=0.075)
LR_ex_2 = lgb.LGBMRegressor(random_state=22, n_estimators=1000, max_depth=61, learning_rate=0.075)
LR_ex_3 = lgb.LGBMRegressor(random_state=2, n_estimators=1000, max_depth=61, learning_rate=0.075)


estimators2 = [     ('lr1', LR_ex_1),
    ('lr2', LR_ex_2),
     ('lr3', LR_ex_3)

]
reg2 = StackingRegressor(
    estimators=estimators2,
    cv = 5,
     #final_estimator=RandomForestRegressor(n_estimators=10,
                                           #random_state=42)
)
reg2.fit(train_x, train_y_ex)

StackingRegressor(cv=5,
                  estimators=[('lr1',
                               LGBMRegressor(learning_rate=0.075, max_depth=61,
                                             n_estimators=1000,
                                             random_state=42)),
                              ('lr2',
                               LGBMRegressor(learning_rate=0.075, max_depth=61,
                                             n_estimators=1000,
                                             random_state=22)),
                              ('lr3',
                               LGBMRegressor(learning_rate=0.075, max_depth=61,
                                             n_estimators=1000,
                                             random_state=2))])

In [17]:
test_ex = reg2.predict(test_x)

In [18]:
test_ex.max()

0.7642979368912256

In [19]:
submission.Reorg_g = test_g
submission.Reorg_ex = test_ex

In [20]:
submission

,Reorg_g,Reorg_ex
index,,
test_0,0.350062,0.481859
test_1,0.880274,0.590141
test_2,0.421668,0.300097
test_3,0.451560,0.371296
test_4,0.341571,0.433553
...,...,...
test_452,0.183908,0.262319
test_453,0.191882,0.271128
test_454,0.338312,0.313922


In [21]:
submission.to_csv(path + 'stack.csv')

In [50]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb

LR = MultiOutputRegressor(lgb.LGBMRegressor(random_state=42, n_estimators=1000, max_depth=61, learning_rate=0.075)).fit(train_x, train_y)

In [51]:
pred0 = LR.predict(test_x)

In [53]:
pred0.max()

0.9017997109847191

In [58]:
submission[['Reorg_g','Reorg_ex']] = pred

In [59]:
submission

,Reorg_g,Reorg_ex
index,,
test_0,0.398319,0.399333
test_1,0.776101,0.517195
test_2,0.378620,0.279672
test_3,0.454643,0.341835
test_4,0.345880,0.401776
...,...,...
test_452,0.211775,0.258755
test_453,0.215280,0.276256
test_454,0.283017,0.289056


In [60]:
submission.to_csv('/content/drive/MyDrive/Colab Notebooks/Dacon/samsung_ai/data/pred_esm2.csv')

In [54]:
pred1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Dacon/samsung_ai/data/pred_10242.csv', index_col = 0)[['Reorg_g', 'Reorg_ex']]
pred2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Dacon/samsung_ai/data/pred_10243.csv', index_col = 0)[['Reorg_g', 'Reorg_ex']]
pred = (pred0 + pred1 + pred2)/3

In [ ]:
class ChemBERTDataset(torch.utils.data.Dataset):
    def __init__(self, data, label_df):
        self.data = data
        self.label = label_df

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.data.items()}
        item['label'] = torch.tensor([self.label[idx]], dtype = torch.float)
        
        return item

    def __len__(self):
        return len(self.data.input_ids)

In [ ]:
model_name = 'DeepChem/ChemBERTa-77M-MLM'
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
tokenizer = AutoTokenizer.from_pretrained(model_name)

config = AutoConfig.from_pretrained(model_name)
config.num_labels = 1
model =  AutoModelForSequenceClassification.from_config(config)

loading configuration file https://huggingface.co/DeepChem/ChemBERTa-77M-MLM/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/86913776bc30b1494bfe9d7691873f74d55090fa479847495e81111d9105ca63.4e4a24ae990a84c67b78efe89e60d972a98f7c22ca6e9e86f81b5a5d1da1d1ea
Model config RobertaConfig {
  "_name_or_path": "DeepChem/ChemBERTa-77M-MLM",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.109,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.144,
  "hidden_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 464,
  "is_gpu": true,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 515,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 3,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.21.2",
  "type_vocab_size": 1,
  "use_cache": true,


In [ ]:
from sklearn.metrics import mean_squared_error
def compute_metrics(pred):
  """ validation을 위한 metrics function """
  preds, labels = pred

  mse = mean_squared_error(labels, preds)

  return {
      'mse': mse,
  }

In [ ]:
from sklearn.model_selection import train_test_split, KFold

cv = KFold(n_splits = 5, random_state = 42, shuffle = True)
cv_pred2 = []
for idx, (train_idx, val_idx) in enumerate(cv.split(train)):
  model =  AutoModelForSequenceClassification.from_config(config)
  train_set, eval_set = train.iloc[train_idx],  train.iloc[val_idx]

  tokenized_train = tokenizer(
      list(train_set.SMILES),
      return_tensors="pt",
      #max_length=128, 
      padding=True,
      truncation=True,
      )

  tokenized_eval = tokenizer(
      list(eval_set.SMILES),
      return_tensors="pt",
      #max_length=128,
      padding=True,
      truncation=True,
      )

  tokenized_test = tokenizer(
      list(test.SMILES),
      return_tensors="pt",
      #max_length=128,
      padding=True,
      truncation=True,
      )

  train_dataset = ChemBERTDataset(tokenized_train, train_set['Reorg_ex'])
  eval_dataset = ChemBERTDataset(tokenized_eval, eval_set['Reorg_ex'])

  training_ars = TrainingArguments(
    output_dir='./result',
    num_train_epochs=10,
    per_device_train_batch_size=64, # 32
    learning_rate=5e-4,
    weight_decay=0.1,
    save_total_limit=5,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    logging_strategy='epoch',
    load_best_model_at_end = True,
  )

  trainer = Trainer(
    model=model,
    args=training_ars,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
  )

  trainer.train()

  num_test = len(tokenized_test['input_ids'])
  test_label = torch.zeros(num_test)
  test_dataset = ChemBERTDataset(tokenized_test, test_label)

  dataloader = DataLoader(test_dataset, batch_size= num_test, shuffle=False)

  model.eval()
  output_pred = []

  for i, data in enumerate(tqdm(dataloader)):
      with torch.no_grad():
          outputs = model(
              input_ids=data['input_ids'].to(device),
              attention_mask=data['attention_mask'].to(device),
          )
      logits = outputs[0]
      logits = logits.detach().cpu().numpy()

      output_pred.append(logits)
  cv_pred2.append(output_pred[0])     



PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 14525
  Num Epochs = 10
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 2270


Epoch,Training Loss,Validation Loss,Mse
1,0.133600,0.097822,0.097822
2,0.104000,0.095602,0.095602
3,0.098700,0.095938,0.095938
4,0.094700,0.100200,0.100200
5,0.092700,0.107869,0.107869
6,0.090700,0.094788,0.094788
7,0.087300,0.092221,0.092221
8,0.084800,0.093808,0.093808
9,0.079100,0.096088,0.096088
10,0.075100,0.096932,0.096932


***** Running Evaluation *****
  Num examples = 3632
  Batch size = 8
Saving model checkpoint to ./result/checkpoint-227
Configuration saved in ./result/checkpoint-227/config.json
Model weights saved in ./result/checkpoint-227/pytorch_model.bin
tokenizer config file saved in ./result/checkpoint-227/tokenizer_config.json
Special tokens file saved in ./result/checkpoint-227/special_tokens_map.json
Deleting older checkpoint [result/checkpoint-2043] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 3632
  Batch size = 8
Saving model checkpoint to ./result/checkpoint-454
Configuration saved in ./result/checkpoint-454/config.json
Model weights saved in ./result/checkpoint-454/pytorch_model.bin
tokenizer config file saved in ./result/checkpoint-454/tokenizer_config.json
Special tokens file saved in ./result/checkpoint-454/special_tokens_map.json
Deleting older checkpoint [result/checkpoint-2724] due to args.save_total_limit
***** Running Evaluation *****
  Num examp

Epoch,Training Loss,Validation Loss,Mse
1,0.132100,0.098118,0.098118
2,0.103800,0.098954,0.098954
3,0.099600,0.098765,0.098765
4,0.096000,0.097221,0.097221
5,0.092800,0.095478,0.095478
6,0.090000,0.094606,0.094606
7,0.087900,0.100022,0.100022
8,0.084600,0.095586,0.095586
9,0.081100,0.097437,0.097437
10,0.076800,0.099160,0.099160


***** Running Evaluation *****
  Num examples = 3632
  Batch size = 8
Saving model checkpoint to ./result/checkpoint-227
Configuration saved in ./result/checkpoint-227/config.json
Model weights saved in ./result/checkpoint-227/pytorch_model.bin
tokenizer config file saved in ./result/checkpoint-227/tokenizer_config.json
Special tokens file saved in ./result/checkpoint-227/special_tokens_map.json
Deleting older checkpoint [result/checkpoint-1362] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 3632
  Batch size = 8
Saving model checkpoint to ./result/checkpoint-454
Configuration saved in ./result/checkpoint-454/config.json
Model weights saved in ./result/checkpoint-454/pytorch_model.bin
tokenizer config file saved in ./result/checkpoint-454/tokenizer_config.json
Special tokens file saved in ./result/checkpoint-454/special_tokens_map.json
Deleting older checkpoint [result/checkpoint-1589] due to args.save_total_limit
***** Running Evaluation *****
  Num examp

...

***** Running Evaluation *****
  Num examples = 3631
  Batch size = 8
Saving model checkpoint to ./result/checkpoint-227
Configuration saved in ./result/checkpoint-227/config.json
Model weights saved in ./result/checkpoint-227/pytorch_model.bin
tokenizer config file saved in ./result/checkpoint-227/tokenizer_config.json
Special tokens file saved in ./result/checkpoint-227/special_tokens_map.json
Deleting older checkpoint [result/checkpoint-1362] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 3631
  Batch size = 8
Saving model checkpoint to ./result/checkpoint-454
Configuration saved in ./result/checkpoint-454/config.json
Model weights saved in ./result/checkpoint-454/pytorch_model.bin
tokenizer config file saved in ./result/checkpoint-454/tokenizer_config.json
Special tokens file saved in ./result/checkpoint-454/special_tokens_map.json
Deleting older checkpoint [result/checkpoint-1589] due to args.save_total_limit
***** Running Evaluation *****
  Num examp

Epoch,Training Loss,Validation Loss,Mse
1,0.133900,0.100265,0.100265
2,0.103300,0.098987,0.098987
3,0.099800,0.100915,0.100915
4,0.094900,0.098499,0.098499
5,0.093500,0.095992,0.095992
6,0.090800,0.095940,0.095940
7,0.087600,0.095472,0.095472
8,0.084900,0.098248,0.098248
9,0.081100,0.097737,0.097737
10,0.076600,0.097385,0.097385


***** Running Evaluation *****
  Num examples = 3631
  Batch size = 8
Saving model checkpoint to ./result/checkpoint-227
Configuration saved in ./result/checkpoint-227/config.json
Model weights saved in ./result/checkpoint-227/pytorch_model.bin
tokenizer config file saved in ./result/checkpoint-227/tokenizer_config.json
Special tokens file saved in ./result/checkpoint-227/special_tokens_map.json
Deleting older checkpoint [result/checkpoint-1362] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 3631
  Batch size = 8
Saving model checkpoint to ./result/checkpoint-454
Configuration saved in ./result/checkpoint-454/config.json
Model weights saved in ./result/checkpoint-454/pytorch_model.bin
tokenizer config file saved in ./result/checkpoint-454/tokenizer_config.json
Special tokens file saved in ./result/checkpoint-454/special_tokens_map.json
Deleting older checkpoint [result/checkpoint-1589] due to args.save_total_limit
***** Running Evaluation *****
  Num examp

Epoch,Training Loss,Validation Loss,Mse
1,0.136200,0.110974,0.110974
2,0.104800,0.098886,0.098886
3,0.099900,0.094113,0.094113
4,0.096700,0.094591,0.094591
5,0.094000,0.093843,0.093843
6,0.090700,0.096051,0.096051
7,0.088700,0.096482,0.096482
8,0.084900,0.093311,0.093311
9,0.080800,0.092934,0.092934
10,0.077200,0.094983,0.094983


***** Running Evaluation *****
  Num examples = 3631
  Batch size = 8
Saving model checkpoint to ./result/checkpoint-227
Configuration saved in ./result/checkpoint-227/config.json
Model weights saved in ./result/checkpoint-227/pytorch_model.bin
tokenizer config file saved in ./result/checkpoint-227/tokenizer_config.json
Special tokens file saved in ./result/checkpoint-227/special_tokens_map.json
Deleting older checkpoint [result/checkpoint-1362] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 3631
  Batch size = 8
Saving model checkpoint to ./result/checkpoint-454
Configuration saved in ./result/checkpoint-454/config.json
Model weights saved in ./result/checkpoint-454/pytorch_model.bin
tokenizer config file saved in ./result/checkpoint-454/tokenizer_config.json
Special tokens file saved in ./result/checkpoint-454/special_tokens_map.json
Deleting older checkpoint [result/checkpoint-1589] due to args.save_total_limit
***** Running Evaluation *****
  Num examp

In [ ]:
submission.Reorg_ex = np.mean(cv_pred2, axis = 0)

In [ ]:
submission

,Reorg_g,Reorg_ex
index,,
test_0,0.404351,0.391127
test_1,0.699554,0.545267
test_2,0.564711,0.476591
test_3,0.762356,0.762603
test_4,0.324132,0.284140
...,...,...
test_452,0.274878,0.224002
test_453,0.283318,0.220352
test_454,0.283416,0.228271


In [ ]:
submission.to_csv(path + 'pred1.csv')

In [ ]:
train_dataset = ChemBERTDataset(tokenized_train, train_dataset['Reorg_g'])
eval_dataset = ChemBERTDataset(tokenized_eval, eval_dataset['Reorg_g'])

In [ ]:
data = next(iter(eval_dataset))
data

{'input_ids': tensor([12, 16, 19, 15, 20, 15, 15, 17, 39, 16, 22, 16, 17, 39, 34, 18, 16, 17,
         22, 19, 18, 19, 18, 15, 15, 17, 54, 18, 15, 20, 19, 16, 16, 17, 23, 18,
         22, 19, 13,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0

In [ ]:
train_loader = DataLoader(train_dataset, batch_size = 64)
eval_loader = DataLoader(eval_dataset, batch_size = 32)

In [ ]:
data= next(iter(train_loader))
data

{'input_ids': tensor([[12, 19, 22,  ...,  0,  0,  0],
         [12, 16, 16,  ...,  0,  0,  0],
         [12, 16, 23,  ...,  0,  0,  0],
         ...,
         [12, 16, 16,  ...,  0,  0,  0],
         [12, 16, 39,  ...,  0,  0,  0],
         [12, 23, 16,  ...,  0,  0,  0]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'label': tensor([[0.9420],
         [0.0754],
         [1.0972],
         [0.7301],
         [1.0212],
         [0.9012],
         [0.2736],
         [0.3856],
         [0.7078],
         [0.1532],
         [0.2484],
         [0.1809],
         [0.3691],
         [0.3106],
         [0.9631],
         [0.1434],
         [0.3478],
         [0.5353],
         [1.4660],
         [0.7554],
         [0.5365],
         [1.0584],
         [0.7673],
         [0.1409],
         [0.5407

In [ ]:
optimizer = torch.optim.Adam(params =  model.parameters(), lr= 1e-4)

def loss_fn(outputs, targets):
    return torch.nn.MSELoss()(outputs, targets)

def trainer(epoch):
    model.to(device)
    model.train()
    min_loss = np.inf
    for e in range(epoch):
      for _, (data) in enumerate(train_loader, 0):
        ids = data['input_ids'].to(device, dtype = torch.long)
        mask = data['attention_mask'].to(device, dtype = torch.long)

        targets = data['label'].to(device, dtype = torch.float)

        outputs = model(ids, mask)

        optimizer.zero_grad()
        loss = loss_fn(outputs.logits, targets)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
      print(f'Epoch: {e+1}, Loss:  {loss.item()}')
      if loss < min_loss:
        min_loss = loss
        val_loss = validation()
        print(f'Epoch: {e+1}, Val_Loss:  {val_loss.item()}')
      

def validation():
    model.eval()

    fin_outputs=[]
    with torch.no_grad():
        for _, (data) in enumerate(eval_loader, 0):
            ids = data['input_ids'].to(device, dtype = torch.long)
            mask = data['attention_mask'].to(device, dtype = torch.long)

            targets = data['label'].to(device, dtype = torch.float)
            outputs = model(ids, mask)
            loss = loss_fn(outputs.logits, targets)

    return loss


In [ ]:
trainer(10)

Epoch: 1, Loss:  0.13384997844696045
Epoch: 1, Val_Loss:  0.12674137949943542
Epoch: 2, Loss:  0.12413898855447769
Epoch: 2, Val_Loss:  0.11923918128013611
Epoch: 3, Loss:  0.11315642297267914
Epoch: 3, Val_Loss:  0.09465809911489487
Epoch: 4, Loss:  0.10693023353815079
Epoch: 4, Val_Loss:  0.09483005851507187
Epoch: 5, Loss:  0.10187987983226776
Epoch: 5, Val_Loss:  0.096208855509758
Epoch: 6, Loss:  0.09389141947031021
Epoch: 6, Val_Loss:  0.10159632563591003


KeyboardInterrupt: ignored

In [ ]:
num_test = len(tokenized_test['input_ids'])
test_label_df = pd.DataFrame(np.zeros(num_test), columns = ['Reorg_g'])
test_label_df['Reorg_ex'] = 0.0

In [ ]:
test_label_df

,Reorg_g,Reorg_ex
0,0.0,0.0
1,0.0,0.0
2,0.0,0.0
3,0.0,0.0
4,0.0,0.0
...,...,...
452,0.0,0.0
453,0.0,0.0
454,0.0,0.0
455,0.0,0.0


In [ ]:
test_dataset = ChemBERTDataset(tokenized_test, test_label_df)
test_loader = DataLoader(test_dataset)

In [ ]:
def test():
    model.eval()

    fin_outputs=[]
    with torch.no_grad():
        for _, (data,label) in enumerate(test_loader, 0):
            ids = data['input_ids'].to(device, dtype = torch.long)
            mask = data['attention_mask'].to(device, dtype = torch.long)

            outputs = model(ids, mask)
            fin_outputs.extend(outputs.logits.cpu().detach().numpy().tolist())
    return fin_outputs

In [ ]:
pred = test()

In [ ]:
training_ars = TrainingArguments(
    output_dir='./result',
    num_train_epochs=7,
    per_device_train_batch_size=32, # 64
    learning_rate=5e-5,
    weight_decay=0.1,
    save_total_limit=5,
    save_steps=500,
    evaluation_strategy='steps',
    eval_steps = 500,
    load_best_model_at_end = True,
)

trainer = Trainer(
    model=model,
    args=training_ars,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 2
  Num Epochs = 7
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 7


Step,Training Loss,Validation Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=7, training_loss=0.6849952425275531, metrics={'train_runtime': 0.3898, 'train_samples_per_second': 35.914, 'train_steps_per_second': 17.957, 'total_flos': 66772565160.0, 'train_loss': 0.6849952425275531, 'epoch': 7.0})

In [ ]:
num_test = len(tokenized_test['input_ids'])
test_label = torch.zeros(num_test)
test_dataset = ChemBERTDataset(tokenized_test, test_label)

dataloader = DataLoader(test_dataset, batch_size= num_test, shuffle=False)

model.eval()
output_pred = []

for i, data in enumerate(tqdm(dataloader)):
    with torch.no_grad():
        outputs = model(
            input_ids=data['input_ids'].to(device),
            attention_mask=data['attention_mask'].to(device),
        )
    logits = outputs[0]
    logits = logits.detach().cpu().numpy()

    output_pred.append(logits) 

  0%|          | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
100%|██████████| 1/1 [00:00<00:00,  1.83it/s]


In [ ]:
submission.Reorg_ex = output_pred[0]

In [ ]:
submission.to_csv(path + 'pred.csv')

In [ ]:


ATOM_LIST = ['C', 'N', 'O', 'S', 'F', 'Si', 'P', 'Cl', 'Br', 'unk']
def onek_encoding(x, allowable_set):
    if x not in allowable_set:                                                                                                                                               
        raise Exception('input {0} not in allowable set{1}:'.format(x, allowable_set))
    return list(map(lambda s: x == s, allowable_set))

def onek_encoding_unk(x, allowable_set):
    if x not in allowable_set:
        x = allowable_set[-1]
    return list(map(lambda s: x == s, allowable_set))

def atom_features(atom):
        return np.array(onek_encoding_unk(atom.GetSymbol(), ATOM_LIST)
                        + onek_encoding_unk(atom.GetDegree(), [0, 1, 2, 3, 4, 5])
                        + onek_encoding_unk(atom.GetExplicitValence(), [1, 2, 3, 4, 5, 6])
                        + onek_encoding_unk(atom.GetImplicitValence(), [0, 1, 2, 3, 4, 5])
                        + [atom.GetIsAromatic()], dtype=np.float32)

In [ ]:
atom = []
for a in mol.GetAtoms():
  atom.append(atom_features(a).reshape(1, -1))
  #except: atom.append(np.zeros(82).reshape(1, -1))
atom = np.vstack(atom)

In [ ]:
atom[-1]

array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [ ]:
a.GetIsAromatic()

False